Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [3]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [4]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])
# the pipeline stitches together fit, transform, predict methods.

In [5]:
#fine-tunning pipeline components (i.e. vectorizer, classifier)
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   55.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [6]:
grid_search.best_score_

0.8903150525087514

In [7]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [59]:
'''Base stuff'''
import os
import re

import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import spacy

from collections import Counter

''' Gensim Stuff '''
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import SnowballStemmer

### Load Competition Data

In [60]:
import pandas as pd
train = pd.read_csv('./ds7-which-whisky/train.csv')
test = pd.read_csv('./ds7-which-whisky/test.csv')

In [61]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.80, random_state=69)

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [62]:
''' Cleaning up the text and preparing for NLP '''
STOPWORDS = set(STOPWORDS).union(set(['the', 'aa', 'aaa', 'aaaaaa' ])) # will update as I go further along


def tokenize(text): 
    return [token for token in simple_preprocess(text) if token not in STOPWORDS and len(token) >=3]

''' Also throwing in some regex cleaning just for good measure '''
def clean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.replace('\n', '')
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [63]:
train.head()

,id,description,category
2338,3726,Or ‘Fumabat Porticus’ as it should be called—s...,1
295,499,"From a single barrel, but the barrel number is...",2
250,396,Another Lock Stock & Barrel all rye-grain whis...,4
2357,3748,"Rich on the nose, with damp tweed, malt, and a...",1
1008,1610,"Highly fragrant and citric, mixing dried and s...",1


In [64]:
train['tokens'] = train['description'].apply(tokenize, clean)
val['tokens'] = val['description'].apply(tokenize, clean)
test['tokens'] = test['description'].apply(tokenize, clean)

In [65]:
train['tokens'].head()

2338    [fumabat, porticus, called, smoky, whisky, por...
295     [single, barrel, barrel, number, unfortunately...
250     [lock, stock, barrel, rye, grain, whisky, pot,...
2357    [rich, nose, damp, tweed, malt, suggestion, ea...
1008    [highly, fragrant, citric, mixing, dried, swee...
Name: tokens, dtype: object

### Define Pipeline Components

In [66]:
from sklearn.linear_model import SGDClassifier

vect = TfidfVectorizer(stop_words='english')
clf = SGDClassifier(alpha=0.0001, 
                    average=False, 
                    early_stopping=False,
                    epsilon=0.1,
                    eta0=0.01, 
                    fit_intercept=True, 
                    l1_ratio=0.15,
                    learning_rate='adaptive', 
                    loss='squared_loss', 
                    max_iter=1000,
                    n_iter_no_change=42, 
                    penalty='l2', 
                    power_t=0.25, 
                    random_state=69,
                    shuffle=True, 
                    tol=0.001, 
                    validation_fraction=0.1, 
                    verbose=0,
                    warm_start=False)

pipe = Pipeline([('vect', vect), ('clf', clf)])

# Fit Pipeline
pipe.fit(train['tokens'].astype(str), train['category'])

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...om_state=69, shuffle=True,
       tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False))])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [67]:
# Declaring some parameters to fine tune
parameters = {
    #'clf__alpha' : (.0001),
    #'clf__max_iter' : (1000),
    'clf__n_iter_no_change' : (45, 50),
    #'clf__learning_rate' : ('constant', 'optimal', 'invscaling', 'adaptive'),
    #'clf__tol' : (.001, .002, .01, .1),
    #'clf__loss' : ('squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'),
    #'clf__penalty' : ('none', 'l2', 'l1', 'elasticnet')
    #'clf__n_estimators' : (10, 20)
    

}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train['tokens'].astype(str), train['category'])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    9.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...om_state=69, shuffle=True,
       tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'clf__n_iter_no_change': (45, 50)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [68]:
test.head()

,id,description,tokens
0,955,"Think carnival aromas—the good ones, anyway—me...","[think, carnival, aromas, good, ones, meeting,..."
1,3532,"A blend of three bourbons, between 6 and 12 ye...","[blend, bourbons, years, old, rye, wheat, reve..."
2,1390,"The nose is focused on cereal, hints of fresh ...","[nose, focused, cereal, hints, fresh, ripe, ch..."
3,1024,Swiss-based Chapter 7 released this 19 year ol...,"[swiss, based, chapter, released, year, old, s..."
4,1902,Valkyrie replaces the current Dark Origins exp...,"[valkyrie, replaces, current, dark, origins, e..."


### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [69]:
# Predictions on test sample
pred = grid_search.predict(val['tokens'].astype(str))

In [70]:
pred

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 2, 2, 3, 1, 1, 4, 1, 1, 1,
       1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 2, 2, 3, 1, 1, 3, 1, 1, 4, 1, 1, 2, 3, 1, 1, 3, 3,
       2, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 2, 1, 1, 2, 2, 1, 4, 1, 1, 1, 4, 3, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 4, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2,
       3, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 2, 2, 2, 2, 3,
       1, 1, 1, 2, 1, 1, 1, 4, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 1, 4, 1,
       1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 4, 4, 2, 2, 2, 4, 2, 1, 1, 4, 1, 2, 2, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 3, 1, 1, 1, 4, 2, 1, 1, 1, 1, 2, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1,
       1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 3, 3, 1, 2, 2, 4, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2,
       2, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1,

In [71]:
from sklearn.metrics import accuracy_score

In [72]:
accuracy_score(val['category'], pred)

0.9266409266409267

In [73]:
pred = grid_search.predict(test['tokens'].astype(str))

In [74]:
submission = pd.DataFrame({'id':test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [75]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,4
3,1024,1
4,1902,1


In [76]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./ds7-which-whisky/submission2.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [9]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [10]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0]
}

In [12]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [13]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:   23.7s finished
/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [14]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [19]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [22]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [ ]:
# Apply to your Dataset

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on